# Модуль 13: Интеграция с R, Python и SQL

**Тема: Использование внешних вычислений в Tableau
Цель**: Научить подключать R, Python и пользовательские SQL-запросы к Tableau, чтобы выполнять сложные вычисления, которые выходят за рамки встроенных функций Tableau.

## Теория
**Что такое интеграция с R, Python и SQL?**
Tableau умеет дружить с R, Python и SQL, чтобы делать расчёты, которые сам не может. Например, Python — для машинного обучения, R — для статистики, а SQL — для сложных запросов в базе данных. Это как позвать специалистов, когда Tableau не хватает силёнок.
**Как работает интеграция?**  
* **R и Python**: Ты пишешь код в вычисляемом поле Tableau с помощью функций SCRIPT_* (например, SCRIPT_REAL для чисел). Tableau отправляет данные в R или Python, выполняет код и возвращает результат.  
* **SQL**: Пишешь пользовательский SQL-запрос в Data Source, чтобы получить данные в нужном виде, или используешь функции RAWSQL_* в формулах для запросов прямо в базе.

⠀**Как настроить R и Python?**  
* Установи R (например, RStudio) или Python (например, Anaconda).  
* Настрой сервер:  
  * Для R: Установи пакет Rserve (install.packages("Rserve")) и запусти его (library(Rserve); Rserve()).  
  * Для Python: Установи TabPy (pip install tabpy) и запусти сервер (tabpy).
* В Tableau: File → Settings and Performance → Manage External Service Connection → выбери Rserve (порт 6311) или TabPy (порт 9004).  
* Создай вычисляемое поле с SCRIPT_* и вставь код.

⠀**Как использовать пользовательский SQL?**  
* При подключении к базе в Data Source выбери **New Custom SQL** и напиши запрос (например, SELECT Category, SUM(Sales) FROM table GROUP BY Category).  
* Или создай вычисляемое поле с RAWSQL_* (например, RAWSQL_REAL) для выполнения запросов в базе.  
* Убедись, что база поддерживает SQL (например, MySQL, PostgreSQL, SQL Server).

⠀**Как интеграция работает с фильтрами?**  
* **Фильтры в рабочем листе**: Фильтры Tableau (например, [Region] = "West") применяются *до* отправки данных в R, Python или SQL. Если ты фильтруешь только West, скрипты или запросы будут работать только с этими данными.  
* **Фильтры на уровне Data Source**: Эти фильтры (например, [Order Date] = 2025) убирают данные ещё до Tableau, так что R, Python и SQL видят только отфильтрованные данные.  
* **Совет**: Проверяй фильтры в Data Source, чтобы понимать, какие данные отправляются в скрипты или запросы.

⠀**Сравнение R, Python, SQL и формул Tableau**  
| **Что сравниваем** | **R, Python, SQL** | **Формулы Tableau** |
|:-:|:-:|:-:|
| **Что делает?** | Сложные расчёты: статистика (R), машинное обучение (Python), запросы (SQL). | Простые вычисления: суммы, доли, ранги, LOD (см. Модули 8–12). |
| **Когда использовать?** | Когда Tableau не хватает (например, регрессия, кластеризация, сложные JOIN). | Для стандартных задач (например, фильтры, сегменты, топ-N). |
| **Сложность** | Нужны знания R, Python или SQL и настройка серверов. | Простые, работают сразу в Tableau. |
| **Пример** | Прогноз продаж (Python), корреляция (R), сложный JOIN (SQL). | Доля продаж региона или топ-5 клиентов. |
**Почему это важно?**
R, Python и SQL — это как супергерои для Tableau. Они позволяют делать сложные анализы, которые встроенные функции не потянут: прогнозировать продажи, кластеризовать клиентов или обрабатывать огромные данные в базе. Это делает твои дашборды мощными и открывает двери для продвинутого анализа.

## Примеры с таблицами
**Исходная таблица базы данных** (Superstore):  
| Order ID | Category | Region | Customer Name | Order Date | Sales | Profit |
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
| 1 | Technology | West | Anna | 2025-01-01 | 1000 | 200 |
| 2 | Technology | East | Anna | 2025-02-25 | 2000 | 400 |
| 3 | Furniture | West | Bob | 2025-01-15 | 1500 | 300 |
| 4 | Furniture | East | Bob | 2025-03-01 | 500 | -50 |
| 5 | Technology | West | Charlie | 2025-01-10 | 800 | 100 |
* **Python: Кластеризация клиентов по продажам**  
  * **Условие**: Хотим разделить клиентов на 2 кластера по продажам с помощью k-means в Python.  
  * **Что делаем**:  
    * Настраиваем TabPy.  
    * Создаём вычисляемое поле:  
    SCRIPT_INT("
    from sklearn.cluster import KMeans
    import numpy as np
    sales = __arg1
    kmeans = KMeans(n_clusters=2, random_state=0).fit(np.array(sales).reshape(-1, 1))
    return kmeans.labels_.tolist()
    ", SUM([Sales])) // Кластер клиентов
  * **Как работает**: Tableau отправляет суммы продаж клиентов в Python, выполняет k-means и возвращает номер кластера (0 или 1).  
  * **Пример**: Anna (3000), Bob (2000), Charlie (800).  
**Итоговая таблица** (визуализация с [Customer Name]):  
| Customer Name | SUM([Sales]) | Кластер клиентов |
|:-:|:-:|:-:|
| Anna | 3000 | 1 |
| Bob | 2000 | 1 |
| Charlie | 800 | 0 |
**Что получаем**: Anna и Bob в кластере 1 (высокие продажи), Charlie в кластере 0 (низкие). Если в Data Source фильтр [Order Date] = 2025-01, Bob (1500) и Charlie (800) могут попасть в один кластер.

* **R: Корреляция между продажами и прибылью**  
  * **Условие**: Хотим посчитать корреляцию между продажами и прибылью по категориям в R.  
  * **Что делаем**:  
    * Настраиваем Rserve.  
    * Создаём вычисляемое поле:  
    SCRIPT_REAL("
    cor(.arg1, .arg2)
    ", SUM([Sales]), SUM([Profit])) // Корреляция
  * **Как работает**: Tableau отправляет продажи и прибыль по категориям в R, выполняет корреляцию и возвращает число.  
  * **Пример**: Technology (Sales: 3000, Profit: 600), Furniture (Sales: 2000, Profit: 250).  
**Итоговая таблица** (визуализация с [Category]):  
| Category | SUM([Sales]) | SUM([Profit]) | Корреляция |
|:-:|:-:|:-:|:-:|
| Technology | 3000 | 600 | 0.999 |
| Furniture | 2000 | 250 | 0.999 |
**Что получаем**: Корреляция ≈ 0.999 (высокая). Если фильтр Data Source убирает Technology, корреляция может измениться.

* **SQL: Сумма продаж по категориям**  
  * **Условие**: Хотим получить сумму продаж по категориям через пользовательский SQL-запрос.  
  * **Что делаем**:  
    * В Data Source выбираем New Custom SQL:  
    SELECT Category, SUM(Sales) as Total_Sales
    FROM Superstore
    GROUP BY Category
  * **Как работает**: Запрос выполняется в базе, возвращая агрегированные данные, которые Tableau использует как источник.  
  * **Пример**: Данные группируются по категориям.  
**Итоговая таблица** (визуализация с [Category]):  
| Category | Total_Sales |
|:-:|:-:|
| Technology | 3000 |
| Furniture | 2000 |
**Что получаем**: Суммы продаж по категориям. Если в Data Source фильтр [Order Date] = 2025-01, Furniture покажет 1500.
* **RAWSQL: Средняя прибыль на заказ**  
  * **Условие**: Хотим посчитать среднюю прибыль на заказ для каждого клиента через SQL.  
  * **Что делаем**:  
    * Создаём вычисляемое поле:   RAWSQL_REAL("SELECT AVG(Profit) FROM Superstore WHERE Customer_Name = %1", [Customer Name]) // Средняя прибыль на заказ
  * **Как работает**: Tableau отправляет запрос в базу, чтобы посчитать среднюю прибыль для каждого клиента.  
  * **Пример**: Anna (200, 400), Bob (300, -50), Charlie (100).  
**Итоговая таблица** (визуализация с [Customer Name]):  
| Customer Name | SUM([Profit]) | Средняя прибыль на заказ |
|:-:|:-:|:-:|
| Anna | 600 | 300 (600/2) |
| Bob | 250 | 125 (250/2) |
| Charlie | 100 | 100 (100/1) |
**Что получаем**: Средняя прибыль на заказ для каждого клиента.
* **Python с параметром: Прогноз продаж**  
  * **Условие**: Хотим, чтобы пользователь выбирал коэффициент для прогноза продаж в Python.  
  * **Что делаем**:  
    * Создаём параметр "Коэффициент прогноза" (тип: Number, диапазон: 1–2, шаг: 0.1, по умолчанию: 1.5).  
    * Показываем параметр.  
    * Создаём вычисляемое поле:  
    SCRIPT_REAL("
    sales = _arg1
    factor = _arg2[0]
    return [s * factor for s in sales]
    ", SUM([Sales]), [Коэффициент прогноза]) // Прогноз продаж
  * **Как работает**: Пользователь выбирает коэффициент, Python умножает продажи на него и возвращает результат.  
  * **Пример**: Пользователь выбрал 1.5.  
**Итоговая таблица** (визуализация с [Category]):  
| Category | SUM([Sales]) | Прогноз продаж |
|:-:|:-:|:-:|
| Technology | 3000 | 4500 |
| Furniture | 2000 | 3000 |
**Что получаем**: Прогноз продаж с коэффициентом 1.5.
* **SQL с набором: Продажи топ-2 клиентов**  
  * **Условие**: Хотим использовать набор топ-2 клиентов и посчитать их продажи через SQL.  
  * **Что делаем**:  
    * Создаём набор "Топ-2 клиентов" (Top 2 by SUM([Sales])).  
    * Создаём вычисляемое поле:  
    RAWSQL_REAL("
    SELECT SUM(Sales)
    FROM Superstore
    WHERE Customer_Name = %1
    AND Customer_Name IN (SELECT Customer_Name FROM Superstore
    GROUP BY Customer_Name
    ORDER BY SUM(Sales) DESC LIMIT 2)
    ", [Customer Name]) // Продажи топ-2 клиентов
  * **Как работает**: SQL проверяет, входит ли клиент в топ-2, и возвращает их продажи.  
  * **Пример**: Anna (3000), Bob (2000).  
**Итоговая таблица** (визуализация с [Customer Name]):  
| Customer Name | SUM([Sales]) | Продажи топ-2 клиентов |
|:-:|:-:|:-:|
| Anna | 3000 | 3000 |
| Bob | 2000 | 2000 |
| Charlie | 800 | NULL |
**Что получаем**: Продажи для Anna и Bob, Charlie — NULL.

## Рекомендации для новичков
* **Проверяй серверы**: Убедись, что Rserve или TabPy запущены, и Tableau подключён к правильному порту (6311 для R, 9004 для TabPy).  
* **Тестируй простой код**: Начни с лёгкого скрипта (например, умножение чисел), чтобы проверить связь с R или Python.  
* **Проверяй фильтры**: Фильтры в рабочем листе и Data Source ограничивают данные, отправляемые в скрипты или SQL. Data Source фильтры (например, [Year] = 2025) влияют на всё.  
* **Обрабатывай NULL**: Используй ZN() в Tableau (например, ZN(SUM([Sales]))) или обработку NULL в R/Python/SQL, чтобы избежать пропусков (см. Модуль 2).  
* **Пиши комментарии**: Добавляй // Прогноз продаж или /* Корреляция */ в формулы, чтобы не запутаться.  
* **Сочетай с параметрами**: Используй параметры (см. Модуль 10) для динамичных скриптов, например, коэффициент прогноза.  
* **Не усложняй**: Если задача решается LOD (Модуль 9) или табличными вычислениями (Модуль 8), не подключай R, Python или SQL.  
* **Тестируй на малых данных**: Сложные скрипты или запросы могут тормозить на больших датасетах, проверяй на маленькой выборке.

## Почему это важно?
R, Python и SQL — это как ракета для Tableau. Они позволяют делать то, что встроенные функции не могут: прогнозировать, кластеризовать, анализировать сложные данные или работать с базой напрямую. Это делает твои дашборды не просто красивыми, а настоящими аналитическими инструментами, которые решают крутые задачи.